In [ ]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [ ]:
def search_orders_sin_factura_func(mes: int) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    
    param_dia_ini = datetime(2024, mes, 1)
    param_dia_fin = datetime(2024, mes + 1, 1) - timedelta(days= 1)

    search_orders_sin_factura = [
        "&", "&",
            ("date_order", ">=", param_dia_ini.strftime('%Y-%m-%d')),
            ("date_order", "<=", param_dia_fin.strftime('%Y-%m-%d')),
        "&",
            ("account_move", "=", False),
            ("amount_total", "!=", 0)
        ]

    return search_orders_sin_factura

In [ ]:
def api_call_pos_orders_func(api_params: dict, search_orders_sin_factura: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    search_orders_sin_factura = search_orders_sin_factura


    pos_doc_fields =[
                    'name',
                    'date_order',
                    'partner_id',
                    'session_id',
                    'refunded_order_ids'
                    ] 

    pos_doc_ids = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'search', [search_orders_sin_factura])
    pos_doc_json = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'read', [pos_doc_ids], {'fields': pos_doc_fields})

    

    return pos_doc_json

In [ ]:
def pos_doc_sin_factura_df_func(pos_doc_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pos_ids = []
    data_pos_refunded_ids = []

    for pos in pos_doc_json:
        new = {}
        new['pos_doc_id'] = pos['id']
        new['pos_doc_name'] = pos['name']
        new['refunded_pos_id'] = pos['refunded_order_ids'][0] if pos['refunded_order_ids'] else pd.NA
        new['pos_doc_date'] = pos['date_order']
        new['partner_id'] = pos['partner_id'][0]
        new['partner_name'] = pos['partner_id'][1]
        new['PdV_session'] = pos['session_id'][1]

        if pos['refunded_order_ids']:
            data_pos_refunded_ids.append(pos['refunded_order_ids'][0])
    
        data_pos_ids.append(new)

    pos_doc_sin_factura_df = pd.DataFrame(data_pos_ids)

    pos_doc_sin_factura_df['pos_doc_date'] = pd.to_datetime(pos_doc_sin_factura_df['pos_doc_date'], format='%Y-%m-%d %H:%M:%S')
    
    return pos_doc_sin_factura_df, data_pos_refunded_ids

In [ ]:
def api_call_account_move_func(api_params: dict, data_pos_refunded_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    fact_doc_fields = [
                    'name',
                    'state',
                    'pos_order_ids'
                    ] 

    fact_doc_ids = models.execute_kw(api_db, uid, api_clave, 'account.move', 'search', [[("pos_order_ids", "in", data_pos_refunded_ids)]])
    fact_doc_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [fact_doc_ids], {'fields': fact_doc_fields})
  
    return fact_doc_json

In [ ]:
def fact_doc_state_df_func(fact_doc_json: list[dict]) -> pd.DataFrame:
    
    data_fact_ids = []

    for fact in fact_doc_json:
        new = {}
        new['fact_doc_id'] = fact['id']
        new['fact_doc_name'] = fact['name']
        new['fact_doc_state'] = fact['state']
        new['pos_doc_id'] = fact['pos_order_ids'][0]

        data_fact_ids.append(new)

    fact_doc_state_df = pd.DataFrame(data_fact_ids)
    fact_doc_state_df['fact_doc_id'] = fact_doc_state_df['fact_doc_id'].astype('Int64')

    return fact_doc_state_df

In [ ]:
api_params = api_params_func()
search_orders_sin_factura = search_orders_sin_factura_func(4)

pos_doc_json = api_call_pos_orders_func(api_params, search_orders_sin_factura)

pos_doc_sin_factura_df, data_pos_refunded_ids = pos_doc_sin_factura_df_func(pos_doc_json)

fact_doc_json = api_call_account_move_func(api_params, data_pos_refunded_ids)
fact_doc_state_df = fact_doc_state_df_func(fact_doc_json)

In [ ]:
fact_doc_state_df

In [ ]:
complete_df1 = pos_doc_sin_factura_df.merge(fact_doc_state_df, how='left', left_on='refunded_pos_id', right_on='pos_doc_id')
complete_df = complete_df1.loc[(complete_df1['fact_doc_state'] != "cancel")].drop(columns=['pos_doc_id_y'])
complete_df

In [ ]:
complete_df1